In [1]:
import re
import random
import math
import numpy as np
import torch
import torch.nn as nn

text = (
       'Hello, how are you? I am Romeo.\n'
       'Hello, Romeo My name is Juliet. Nice to meet you.\n'
       'Nice meet you too. How are you today?\n'
       'Great. My baseball team won the competition.\n'
       'Oh Congratulations, Juliet\n'
       'Thanks you Romeo'
   )

In [2]:
sentences = re.sub("[.,!?\\-]", '', text.lower()).split('\n')  # filter '.', ',', '?', '!'
word_list = list(set(" ".join(sentences).split()))

word_dict = {'[PAD]': 0, '[CLS]': 1, '[SEP]': 2, '[MASK]': 3}
for i, w in enumerate(word_list):
    word_dict[w] = i + 4
    number_dict = {i: w for i, w in enumerate(word_dict)}
    vocab_size = len(word_dict)

token_list = [] #句子列表
for i,sent in enumerate(sentences):
    tmp =[]
    for token in sent.split(" "):
        tmp.append(word_dict.get(token))
    token_list.append(tmp)
print(token_list)

# print(sentences)
# tokens_a_index, tokens_b_index = random.randrange(len(sentences)), random.randrange(len(sentences)) 
# # print(tokens_a_index)
# # print(tokens_b_index)

# tokens_a, tokens_b = sentences[tokens_a_index], sentences[tokens_b_index]
# print(tokens_a,word_dict['[CLS]'])
# input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]
# print(input_ids)

[[26, 22, 13, 24, 9, 28, 20], [26, 20, 23, 11, 15, 7, 6, 5, 12, 24], [6, 12, 24, 8, 22, 13, 24, 16], [4, 23, 19, 21, 14, 17, 25], [18, 27, 7], [10, 24, 20]]


### BERT的输入：
![BERT_input](images/BERT_input.png)

In [3]:
batch_size = 4 #批次大小
max_pred = 5 #一行输入里，对多有几个token要被mask

d_model = 128 # 词嵌入的维度(和位置嵌入，segment嵌入的维度一致) embedding_dim
max_seq_len = 32 # 每样本输入的最大token数
n_segments = 2 # 句子数
d_k = d_v = 64 # dimension of K(=Q), V, 不需要跟词嵌入的维度一致,
# d_v = 68  # d_k,d_v必须相等？不想等意味着什么,dv>dk,dv<dk会发生什么？
n_heads = 8 # number of heads in Multi-Head Attention
n_layers = 6 # number of Encoder Layer
d_ff = 2048 # FeedForward dimension

def make_batch():
    """生成bert的训练数据"""
    batch = []
    positive = negative = 0
    while positive != batch_size/2 or negative != batch_size/2:
        tokens_a_index, tokens_b_index = random.randrange(len(sentences)), random.randrange(len(sentences)) 
        
        #token_list,句子集合
        tokens_a, tokens_b = token_list[tokens_a_index], token_list[tokens_b_index]
        
        #1.输入的token ids
        input_ids = [word_dict['[CLS]']] + tokens_a + [word_dict['[SEP]']] + tokens_b + [word_dict['[SEP]']]

        #2.句子的id，上一句，每个token对应位置都是0，下半句都是1
        segment_ids = [0] * (1 + len(tokens_a) + 1) + [1] * (len(tokens_b) + 1)

        #3. MASK LM 
        # 一个输入里面：15%的wordpiece token会被随机掩盖，
        # 这15%的token中80%用[MASK]这个token来代替，10%用随机的一个词来替换,剩下的10%保持不变
        mask_cnt = max(1, int(round(len(input_ids) * 0.15))) # 15 % of tokens in one sentence
#         print("mask_cnt,input_ids_len:",mask_cnt,len(input_ids))
        n_pred =  min(max_pred, mask_cnt) 
        # 能够被用来mask的位置集合
        cand_maked_pos = [i for i, token in enumerate(input_ids)
                         if token != word_dict['[CLS]'] and token != word_dict['[SEP]']]
#         print("cand_maked_pos:",cand_maked_pos)
        random.shuffle(cand_maked_pos)
#         print("cand_maked_pos shuf:",cand_maked_pos)
        masked_tokens, masked_pos = [], []
        for pos in cand_maked_pos[:n_pred]:
            masked_pos.append(pos)
            masked_tokens.append(input_ids[pos])
            if random.random() < 0.8:  # 80%用[MASK]这个token来代替
                input_ids[pos] = word_dict['[MASK]'] # make mask
            elif random.random() < 0.5:  # 10%用随机的一个词来替换
                index = random.randint(0, vocab_size - 1) # random index in vocabulary
                input_ids[pos] = word_dict[number_dict[index]] # replace

        # Zero Paddings, 其余部分补0
        n_pad = max_seq_len - len(input_ids)
        input_ids.extend([0] * n_pad)
        segment_ids.extend([0] * n_pad)

        # Zero Padding (100% - 15%) tokens
        if max_pred > n_pred:
            n_pad = max_pred - n_pred
            masked_tokens.extend([0] * n_pad)
            masked_pos.extend([0] * n_pad)

        if tokens_a_index + 1 == tokens_b_index and positive < batch_size/2:
            # 这种构造正负样本的方式简单粗暴，仅做演示用，太粗暴了，不过我喜欢
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, True]) # IsNext
            positive += 1
        elif tokens_a_index + 1 != tokens_b_index and negative < batch_size/2:
            batch.append([input_ids, segment_ids, masked_tokens, masked_pos, False]) # NotNext
            negative += 1
            
    return batch


batch = make_batch()
# test
print("input_ids:",batch[0][0],len(batch[0][0]))
print("segment_ids:",batch[0][1],len(batch[0][1]))
print("masked_tokens:",batch[0][2],len(batch[0][2]))
print("masked_pos:",batch[0][3],len(batch[0][3]))
print("label:",batch[0][4])

input_ids: [1, 26, 3, 23, 11, 15, 7, 6, 3, 12, 24, 2, 18, 27, 7, 2, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 32
segment_ids: [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] 32
masked_tokens: [5, 20, 0, 0, 0] 5
masked_pos: [8, 2, 0, 0, 0] 5
label: False


![BERT模型](images/BERT-building-model.webp)

In [9]:
def get_attn_pad_mask(seq_q, seq_k):
    """
    当输入小于max_len时，无效的区域如果为0，过softmax有值，
    为了让这些无效的区域不参与运算，一般是给无效区域加一个很大的负数偏置
    该函数目的，让padding部分变为true，其余为false，用于softmax前的负数填充 。#搜 masked_fill_
    """
    batch_size, len_q = seq_q.size()  #len_q = input_ids's max_seq_len
    batch_size, len_k = seq_k.size()
    # eq(zero) is PAD token
    pad_attn_mask = seq_k.data.eq(0).unsqueeze(1)  # [batch_size,1,len_k(=len_q)], one is masking
    return pad_attn_mask.expand(batch_size, len_q, len_k)  # [batch_size,len_q,len_k]

def gelu(x):
    """等价 torch.nn.functional.gelu"""
    return x * 0.5 * (1.0 + torch.erf(x / math.sqrt(2.0)))


class Embedding(nn.Module):
    def __init__(self):
        super(Embedding, self).__init__()
        self.tok_embed = nn.Embedding(vocab_size, d_model)  # token embedding
        self.pos_embed = nn.Embedding(max_seq_len, d_model)  # position embedding
        self.seg_embed = nn.Embedding(n_segments, d_model)  # segment embedding，上下句子
        self.norm = nn.LayerNorm(d_model) #层归一化

    def forward(self, x, seg):
        seq_len = x.size(1) #x: token ids
        pos = torch.arange(seq_len, dtype=torch.long) #0,1,...sel_len
        pos = pos.unsqueeze(0).expand_as(x)  # (seq_len,) -> (batch_size, seq_len)
        # embedding [batch_size,max_seq_len,d_model]
        embedding = self.tok_embed(x) + self.pos_embed(pos) + self.seg_embed(seg)
        return self.norm(embedding) 

    
class ScaledDotProductAttention(nn.Module):
    def __init__(self):
        super(ScaledDotProductAttention, self).__init__()

    def forward(self, Q, K, V, attn_mask):
        # scores : [batch_size,n_heads,len_q(=len_k),len_k(=len_q)]
        scores = torch.matmul(Q, K.transpose(-1, -2)) / np.sqrt(d_k)
        # 当输入小于max_len时，需要让这些无效的区域不参与运算，一般是给无效区域加一个很大的负数偏置
        scores.masked_fill_(attn_mask, -1e9) # Fills elements of self tensor with value where mask is one.
#         print(scores)
        attn = nn.Softmax(dim=-1)(scores)
#         print(attn)
        context = torch.matmul(attn, V)
#         print("shape:",scores.shape,context.shape,attn.shape)
        #[batch_size,head_cnt,len_q,d_k]
        # scores:  torch.Size([4, 8, 32, 32]) 
        # attn:    torch.Size([4, 8, 32, 32])
        # context: torch.Size([4, 8, 32, 68]) #d_k:32, d_v:68, 一般设定dk=dv，这里测试用 
        # [batch_size,head_cnt,len_q,d_v]
        return scores, context, attn
    
class MultiHeadAttention(nn.Module):
    def __init__(self):
        super(MultiHeadAttention, self).__init__()
        self.W_Q = nn.Linear(d_model, d_k * n_heads)
        self.W_K = nn.Linear(d_model, d_k * n_heads)
        self.W_V = nn.Linear(d_model, d_v * n_heads)
        self.scaled_dot = ScaledDotProductAttention()
        self.fc = nn.Linear(n_heads * d_v, d_model)
        self.layernorm = nn.LayerNorm(d_model)

    def forward(self, Q, K, V, attn_mask):
        # q: [batch_size,seq_len,d_model], k: [batch_size,d_k,d_model], v: [batch_size,d_k,d_model]
        residual, batch_size = Q, Q.size(0)
        
        # print("self.W_Q(Q).size():",self.W_Q(Q).size())
        # torch.Size([4, 32, 512]) d_k * n_heads:512
        
        # 需要将线性变换后多头部分单拆一维出来
        # (batch_size, seq_len, d_model) -proj-> (B, S, D) 
        # -> split-> (B, S, H, W) -trans-> (B, Head_cnt, Seq_len, W)
        
        # q_s: [batch_size,n_heads,len_q,d_k]
        q_s = self.W_Q(Q).view(batch_size, -1, n_heads, d_k).transpose(1,2) 
        # k_s: [batch_size,n_heads,len_k,d_k]
        k_s = self.W_K(K).view(batch_size, -1, n_heads, d_k).transpose(1,2) 
        # v_s: [batch_size,n_heads,len_k,d_v]
        v_s = self.W_V(V).view(batch_size, -1, n_heads, d_v).transpose(1,2)  
        
        # attn_mask : [batch_size,n_heads,len_q,len_k]
        attn_mask = attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1) 
        
        # context: [batch_size,n_heads,len_q,d_v], attn: [batch_size,n_heads,len_q(=len_k),len_k(=len_q)]
        scores, context, attn = self.scaled_dot(q_s, k_s, v_s, attn_mask)
#         print(context.shape)
        # context: [batch_size, len_q, n_heads*d_v]
        context = context.transpose(1, 2).contiguous().view(batch_size, -1, n_heads * d_v) 
        
        output = self.fc(context) 
#         print(output)
        # output: [batch_size,len_q,d_model]
        return self.layernorm(output + residual), attn 


class PoswiseFeedForwardNet(nn.Module):
    def __init__(self):
        super(PoswiseFeedForwardNet, self).__init__()
        # 使用1*1卷积，将d_model维度转为d_ff维度
        self.conv1 = nn.Conv1d(in_channels=d_model, out_channels=d_ff, kernel_size=1) 
        self.conv2 = nn.Conv1d(in_channels=d_ff, out_channels=d_model, kernel_size=1)
        self.layer_norm = nn.LayerNorm(d_model)
        self.activ1 = nn.ReLU()

    def forward(self, inputs):
        residual = inputs # inputs : [batch_size, len_q, d_model]
        output = self.activ1(self.conv1(inputs.transpose(1, 2)))
        output = self.conv2(output).transpose(1, 2)
#         print("PoswiseFeedForwardNet.shape:",output.shape)
        return self.layer_norm(output + residual)


class EncoderLayer(nn.Module):
    def __init__(self):
        super(EncoderLayer, self).__init__()
        self.enc_self_attn = MultiHeadAttention()
        self.pos_ffn = PoswiseFeedForwardNet()

    def forward(self, enc_inputs, enc_self_attn_mask):
        # enc_inputs to same Q,K,V
        enc_outputs, attn = self.enc_self_attn(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask) 
        # enc_outputs: [batch_size x len_q x d_model]
        enc_outputs = self.pos_ffn(enc_outputs) 
        return enc_outputs, attn

    
class BERT(nn.Module):
    def __init__(self):
        super(BERT, self).__init__()
        self.embedding = Embedding()
        self.layers = nn.ModuleList([EncoderLayer() for _ in range(n_layers)])
        self.fc = nn.Linear(d_model, d_model) #？
        self.activ1 = nn.Tanh()
        self.linear = nn.Linear(d_model, d_model)
        self.activ2 = gelu
        self.norm = nn.LayerNorm(d_model)
        self.classifier = nn.Linear(d_model, 2)
        # decoder is shared with embedding layer
        embed_weight = self.embedding.tok_embed.weight
        n_vocab, n_dim = embed_weight.size()
        self.decoder = nn.Linear(n_dim, n_vocab, bias=False)
        self.decoder.weight = embed_weight
        self.decoder_bias = nn.Parameter(torch.zeros(n_vocab))

    def forward(self, input_ids, segment_ids, masked_pos):
        # output : [batch_size, seq_len, d_model]
        output = self.embedding(input_ids, segment_ids)
        enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
        for layer in self.layers:
            output, enc_self_attn = layer(output, enc_self_attn_mask)
        # output : [batch_size, len, d_model], attn : [batch_size, n_heads, d_mode, d_model]
        # it will be decided by first token(CLS)
        h_pooled = self.activ1(self.fc(output[:, 0])) # [batch_size, d_model]
        
        #预测是否为上下句，输出维度：[batch_size, 2]
        logits_clsf = self.classifier(h_pooled) 
        #masked_pos:[batch_size,max_pred,d_model]
        masked_pos = masked_pos[:, :, None].expand(-1, -1, output.size(-1)) # [batch_size, max_pred, d_model]

        # get masked position from final output of transformer.
        h_masked = torch.gather(output, 1, masked_pos) # masking position [batch_size, max_pred, d_model]
        h_masked = self.norm(self.activ2(self.linear(h_masked)))
        logits_lm = self.decoder(h_masked) + self.decoder_bias # [batch_size, max_pred, n_vocab]

        return logits_lm, logits_clsf

In [10]:
# 测试模型里的各个组件
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(*batch))

seq_len = input_ids.size(1)
# print(x.shape)
pos = torch.arange(seq_len, dtype=torch.long)
pos = pos.unsqueeze(0).expand_as(input_ids) 
# print(pos.shape)

pad_attn_mask = input_ids.data.eq(0).unsqueeze(1)
# print(pad_attn_mask)

enc_self_attn_mask = get_attn_pad_mask(input_ids, input_ids)
# print(enc_self_attn_mask)
# print(enc_self_attn_mask.shape) #[batch_size,max_seq_len,max_seq_len]
attn_mask = enc_self_attn_mask.unsqueeze(1).repeat(1, n_heads, 1, 1)  #[batch_size,n_heads,max_seq_len,max_seq_len]
# print(attn_mask.shape)
# print(attn_mask)

embedding = Embedding()
enc_inputs = embedding(input_ids, segment_ids)
# # print(enc_inputs)

mha = MultiHeadAttention()
ret = mha(enc_inputs, enc_inputs, enc_inputs, enc_self_attn_mask)
# # print(ret)

el = EncoderLayer()
ret = el(enc_inputs,enc_self_attn_mask)
# # print(ret)

model = BERT()
logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)

self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])
self.W_Q(Q).size(): torch.Size([4, 32, 512])


In [6]:
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(100):
    optimizer.zero_grad()
    logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
    loss_lm = criterion(logits_lm.transpose(1, 2), masked_tokens) # for masked LM
    loss_lm = (loss_lm.float()).mean()
    loss_clsf = criterion(logits_clsf, isNext) # for sentence classification
    loss = loss_lm + loss_clsf
    if (epoch + 1) % 10 == 0:
        print('Epoch:', '%04d' % (epoch + 1), 'cost =', '{:.6f}'.format(loss))
    loss.backward()
    optimizer.step()

# Predict mask tokens
input_ids, segment_ids, masked_tokens, masked_pos, isNext = map(torch.LongTensor, zip(batch[0]))
print(text)
print([number_dict[w.item()] for w in input_ids[0] if number_dict[w.item()] != '[PAD]'])

logits_lm, logits_clsf = model(input_ids, segment_ids, masked_pos)
logits_lm = logits_lm.data.max(2)[1][0].data.numpy()
print('masked tokens list : ',[pos.item() for pos in masked_tokens[0] if pos.item() != 0])
print('predict masked tokens list : ',[pos for pos in logits_lm if pos != 0])

logits_clsf = logits_clsf.data.max(1)[1].data.numpy()[0]
print('isNext : ', True if isNext else False)
print('predict isNext : ',True if logits_clsf else False)

Epoch: 0010 cost = 5.890497
Epoch: 0020 cost = 3.496031
Epoch: 0030 cost = 2.573384
Epoch: 0040 cost = 2.201141
Epoch: 0050 cost = 2.094465
Epoch: 0060 cost = 2.077641
Epoch: 0070 cost = 2.063523
Epoch: 0080 cost = 2.062627
Epoch: 0090 cost = 2.059888
Epoch: 0100 cost = 2.059503
Hello, how are you? I am Romeo.
Hello, Romeo My name is Juliet. Nice to meet you.
Nice meet you too. How are you today?
Great. My baseball team won the competition.
Oh Congratulations, Juliet
Thanks you Romeo
['[CLS]', 'hello', '[MASK]', 'my', 'name', 'is', 'juliet', 'nice', '[MASK]', 'meet', 'you', '[SEP]', 'oh', 'congratulations', 'juliet', '[SEP]']
masked tokens list :  [5, 20]
predict masked tokens list :  []
isNext :  False
predict isNext :  False


[W NNPACK.cpp:79] Could not initialize NNPACK! Reason: Unsupported hardware.
